#### Importing libraries  

In [1]:
import notebookutils
import pandas as pd
import pyarrow as pa

from deltalake import write_deltalake, DeltaTable
from sempy.fabric import get_notebook_workspace_id, list_workspaces  

#### Obtaining parameters from current workspace and defining paths   

In [2]:
workspace_id = get_notebook_workspace_id()  
workspace_name = list_workspaces().query("Id == @workspace_id")['Name'].iloc[0]
lakehouse_name = 'Lk_IncomeStatement'
lakehouse_path = f'abfss://{workspace_name}@onelake.dfs.fabric.microsoft.com/{lakehouse_name}.Lakehouse'
tables_path = f'{lakehouse_path}/Tables'
files_path = f'{lakehouse_path}/Files/Raw'
 
print(f'Workspace: {workspace_name} | ID: {workspace_id}')
print(f'Tables path: {tables_path}') 
print(f'Files path: {files_path}') 

Workspace: IncomeStatement_DEV | ID: ce398310-c50a-4b25-a3b8-9f733b60c702
Tables path: abfss://IncomeStatement_DEV@onelake.dfs.fabric.microsoft.com/Lk_IncomeStatement.Lakehouse/Tables
Files path: abfss://IncomeStatement_DEV@onelake.dfs.fabric.microsoft.com/Lk_IncomeStatement.Lakehouse/Files/Raw


#### Read each csv file from Raw folder and load to delta  

##### FactIncomeStatement

In [11]:
tb = 'FactIncomeStatement'
df = pd.read_csv(f'{files_path}/{tb}.csv', )

df['CompanyCode'] = df['CompanyCode'].astype('int64') 
df['AccountCode'] = df['AccountCode'].astype('int64')
df['Date']        = pd.to_datetime(df['Date'])
df['Amount']      = df['Amount'].astype('float64')  

arrow_schema = pa.schema([
    pa.field('CompanyCode', pa.int64()),
    pa.field('AccountCode', pa.int64()),
    pa.field('Date',        pa.date32()),   
    pa.field('Amount',      pa.float64()),
])

arr_company = pa.array(df['CompanyCode'])
arr_account = pa.array(df['AccountCode'])
arr_date    = pa.array(df['Date'].dt.date, type=pa.date32())
arr_amount  = pa.array(df['Amount'])

arrow_table = pa.Table.from_arrays(
    [arr_company, arr_account, arr_date, arr_amount],
    schema=arrow_schema
)

write_deltalake(
    f'{tables_path}/{tb}', 
    arrow_table, 
    mode='overwrite'
) 

##### DimDate

In [16]:
tb = 'DimDate'
df = pd.read_csv(f'{files_path}/{tb}.csv', )

df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Year'].astype('int64')
df['Quarter'] = df['Quarter'].astype('int64')
df['MonthName'] = df['MonthName'].astype('string')
df['MonthNameShort'] = df['MonthNameShort'].astype('string')
df['MonthNumber'] = df['MonthNumber'].astype('int64')

arrow_schema = pa.schema([
    pa.field('Date',           pa.date32()),
    pa.field('Year',           pa.int64()),
    pa.field('Quarter',        pa.int64()),
    pa.field('MonthName',      pa.string()),
    pa.field('MonthNameShort', pa.string()),
    pa.field('MonthNumber',    pa.int64()),
])

arr_date        = pa.array(df['Date'].dt.date,   type=pa.date32())
arr_year        = pa.array(df['Year'],           type=pa.int64())
arr_quarter     = pa.array(df['Quarter'],        type=pa.int64())
arr_month_name  = pa.array(df['MonthName'],      type=pa.string())
arr_month_short = pa.array(df['MonthNameShort'], type=pa.string())
arr_month_num   = pa.array(df['MonthNumber'],    type=pa.int64())

arrow_table = pa.Table.from_arrays(
    [arr_date, arr_year, arr_quarter, arr_month_name, arr_month_short, arr_month_num],
    schema=arrow_schema
)

write_deltalake(
    f'{tables_path}/{tb}', 
    arrow_table, 
    mode='overwrite'
) 

##### DimCompany  

In [22]:
tb = 'DimCompany'
df = pd.read_csv(f'{files_path}/{tb}.csv', )

write_deltalake(
    f'{tables_path}/{tb}', 
    df, 
    mode='overwrite'
)

##### DimAccount

In [21]:
tb = 'DimAccount'
df = pd.read_csv(f'{files_path}/{tb}.csv', )

write_deltalake(
    f'{tables_path}/{tb}', 
    df, 
    mode='overwrite'
)